In [2]:
import pandas as pd

from src.worldometer_gateway import WorldOMeterGateway
from src.parser_service import ParserService
from src.parser_service_yesterday import ParserServiceYesterday

In [10]:
def converter(a):
    if a == ' ' or a == '':
        return 0
    if type(a) == str:
        if a == 'N/A':
            return 0
        else:
            return float(a.replace(',','')\
        #                  .replace('\n','')\
                         .replace('+','')\
        #                  .replace(' ','')\
                         .strip())
    else:
        return a

# JHU data

## Generate agg data

In [439]:
df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
focus = df.copy().drop(['Lat','Long'], axis=1).set_index(['Country/Region','Province/State'])
confirm = focus.groupby('Country/Region').sum()

df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
focus = df.copy().drop(['Lat','Long'], axis=1).set_index(['Country/Region','Province/State'])
death = focus.groupby('Country/Region').sum()

df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
focus = df.copy().drop(['Lat','Long'], axis=1).set_index(['Country/Region','Province/State'])
recover = focus.groupby('Country/Region').sum()

for i in [confirm, recover, death]:
    i.columns = pd.to_datetime(confirm.columns)

# confirm.to_csv('data/JH/confirm_agg.csv')
# death.to_csv('data/JH/death_agg.csv')
# recover.to_csv('data/JH/recover_agg.csv')
list(confirm)[-1]

Timestamp('2020-04-26 00:00:00')

## Data correction

In [442]:
confirm.at['Italy', '3/12/20'] = 15113
death.at['Italy', '3/12/20'] = 1016

confirm.at['Spain', '3/12/20'] = 3146
confirm.at['France', '3/12/20'] = 2876
confirm.at['Germany', '3/12/20'] = 2475
confirm.at['Australia', '3/12/20'] = 156
confirm.at['United Kingdom', '3/12/20'] = 590
confirm.at['United Kingdom', '3/15/20'] = 1391
confirm.at['United Kingdom', '3/19/20'] = 3269

confirm.at['New Zealand','3/22/20'] = 66

confirm.at['Serbia', '4/21/20'] = 6890
confirm.at['Serbia', '4/22/20'] = 7114
confirm.at['Serbia', '4/23/20'] = 7276
confirm.at['Serbia', '4/24/20'] = 7483
confirm.at['Serbia', '4/25/20'] = 7779
confirm.at['Serbia', '4/26/20'] = 8042



# output
confirm.to_csv('data/JH/confirm_agg.csv')
death.to_csv('data/JH/death_agg.csv')
recover.to_csv('data/JH/recover_agg.csv')

## Generate daily data

In [443]:
df = confirm.T
for i in list(df):
    df[i] = df[i] - df[i].shift(1)
df.to_csv('data/JH/daily_new_case.csv')
daily_new = df.copy()

df = death.T
for i in list(df):
    df[i] = (df[i] - df[i].shift(1)).fillna(0).astype(int)
df.to_csv('data/JH/daily_new_death.csv')

df = recover.T
for i in list(df):
    df[i] = df[i] - df[i].shift(1)
df.to_csv('data/JH/daily_new_recover.csv')

## Generate for world fire map

In [451]:
set_asia = {'China','Korea, South','Australia','Japan','Malaysia','Philippines','Singapore','New Zealand','Vietnam','Brunei',
            'Cambodia','Mongolia','Fiji','Laos','Papua New Guinea','India','Indonesia','Thailand','Sri Lanka','Bangladesh',
            'Burma','Maldives','Nepal','Bhutan','Timor-Leste'}

set_middle_east = {'Iran','Turkey','Pakistan','Saudi Arabia','United Arab Emirates','Qatar','Egypt','Morocco','Iraq','Bahrain','Kuwait',
                   'Tunisia','Lebanon','Oman','Afghanistan','Jordan','Djibouti','Syria','Libya','Sudan','Somalia','West Bank and Gaza',
                  'Yemen'}

set_europe = {'Spain','Italy','Germany','France','United Kingdom','Switzerland','Belgium','Netherlands','Austria','Portugal',
              'Israel','Sweden','Russia','Norway','Ireland','Czechia','Denmark','Poland','Romania','Luxembourg','Serbia','Finland',
              'Greece','Iceland','Ukraine','Croatia','Estonia','Slovenia','Moldova','Lithuania','Armenia','Hungary','Belarus',
              'Bosnia and Herzegovina','Kazakhstan','Azerbaijan','North Macedonia','Bulgaria','Latvia','Andorra','Slovakia',
              'Uzbekistan','Cyprus','Albania','San Marino','Malta','Kyrgyzstan','Montenegro','Georgia','Liechtenstein','Monaco',
              'Holy See','Kosovo'}

set_africa = {'South Africa','Algeria','Cameroon','Burkina Faso','Cote d\'Ivoire','Mauritius','Nigeria','Senegal','Ghana',
              'Niger','Congo (Kinshasa)','Kenya','Guinea','Rwanda','Madagascar','Uganda','Congo (Brazzaville)','Togo','Ethiopia','Zambia','Mali','Guinea-Bissau',
              'Eritrea','Tanzania','Benin','Gabon','Equatorial Guinea','Namibia','Angola','Liberia','Seychelles','Mozambique',
              'Eswatini','Zimbabwe','Central African Republic','Chad','Cabo Verde','Mauritania','Sierra Leone','Botswana','Gambia',
              'Malawi','Sao Tome and Principe','Burundi','South Sudan','Western Sahara'}

set_america = {'US','Canada','Brazil','Chile','Ecuador','Peru','Mexico','Panama','Dominican Republic','Argentina','Colombia',
               'Costa Rica','Uruguay','Cuba','Honduras','Bolivia','Venezuela','Paraguay','Trinidad and Tobago','Guatemala','El Salvador',
               'Jamaica','Barbados','Bahamas','Guyana','Haiti','Antigua and Barbuda','Dominica','Saint Lucia','Grenada','Suriname',
               'Saint Kitts and Nevis','Belize','Saint Vincent and the Grenadines','Nicaragua'}

set_world = set_asia | set_middle_east | set_europe | set_africa | set_america


In [445]:
fire = daily_new.copy().dropna()
fire = fire[fire.sum().sort_values(ascending=False).index]

fire_asia = fire[[i for i in fire.columns if i in set_asia]]
fire_middle_east = fire[[i for i in fire.columns if i in set_middle_east]]
fire_europe = fire[[i for i in fire.columns if i in set_europe]]
fire_africa = fire[[i for i in fire.columns if i in set_africa]]
fire_america = fire[[i for i in fire.columns if i in set_america]]

fire = fire_asia.join(fire_middle_east).join(fire_africa).join(fire_europe).join(fire_america)
fire.to_clipboard()
fire.to_csv('data/fire map.csv')

# WOM data

## Live data

In [446]:
# Fetch data
worldometer_gateway = WorldOMeterGateway()
parser_service = ParserService()
latest_data = worldometer_gateway.fetch()
df = parser_service.create_df_worldometer(latest_data)
last_updated = parser_service.parse_last_updated(latest_data)

df.set_index('Country/Other', inplace=True)
df.drop('Continent', axis=1, inplace=True)

for i in list(df):
    df[i] = df[i].apply(converter)
    
# write to file
df.to_csv(r'data/worldometer/%s.csv' % last_updated.replace(':','-').replace(',','').replace(' ','_')[14:])
df.to_csv(r'data/worldometer/wom_latest.csv')

print(last_updated)
today = df.copy()

Last updated: April 27, 2020, 13:52 GMT


## Yesterday data

In [447]:
# Fetch data
worldometer_gateway = WorldOMeterGateway()
parser_service = ParserService()
latest_data = worldometer_gateway.fetch()
df = parser_service.create_df_worldometer_yesterday(latest_data)
last_updated = parser_service.parse_last_updated(latest_data)

df.set_index('Country/Other', inplace=True)
df.drop('Continent', axis=1, inplace=True)

for i in list(df):
    df[i] = df[i].apply(converter)
    
# write to file
date = last_updated.split(':')[1].split(',')[0].strip()
df.to_csv(r'data/worldometer/daily_summary_%s.csv' % date)

print(last_updated)
yesterday = df.copy()

Last updated: April 27, 2020, 13:52 GMT


## Timezone issude correct

In [448]:
for country in today.index:
    if today.at[country, 'New Cases'] == 0:
        today.at[country, 'New Cases'] = yesterday.at[country, 'New Cases']
today.to_csv(r'data/worldometer/wom_latest.csv')

In [449]:
increased = pd.DataFrame()
for country in today.sort_values(['Total Cases'], ascending=False).index:
    if today.at[country, 'New Cases'] > yesterday.at[country, 'New Cases'] and yesterday.at[country, 'New Cases'] > 100:
        increased = increased.append([[country, today.at[country, 'New Cases'], yesterday.at[country, 'New Cases'], round(today.at[country, 'New Cases']/yesterday.at[country, 'New Cases']*100 -100, 2)]])
increased.columns = ['Country','New Case','Yesterday Case','Growth Percentage']
increased = increased.set_index('Country').sort_values('Growth Percentage', ascending=False).head(30)
increased

,New Case,Yesterday Case,Growth Percentage
Country,,,
Panama,241.0,200.0,20.50
Bangladesh,497.0,418.0,18.90
Finland,119.0,101.0,17.82
Kuwait,213.0,183.0,16.39
Saudi Arabia,1289.0,1223.0,5.40
Qatar,957.0,929.0,3.01
Kazakhstan,118.0,116.0,1.72


In [450]:
pos_rate = pd.DataFrame()
for country in today.sort_values(['Total Cases'], ascending=False).index:
    if today.at[country,'Total Tests'] > yesterday.at[country,'Total Tests']:
        today_test = int(today.at[country,'Total Tests'] - yesterday.at[country,'Total Tests'])
        today_pos_rate = round(today.at[country,'New Cases'] / today_test * 100, 2)
        pos_rate = pos_rate.append([[
            country,
            int(today.at[country,'New Cases']),
            today_test,
            today_pos_rate
        ]]) 
pos_rate.columns=['Country','New Case','New Test','Positive Percentage']
pos_rate = pos_rate.set_index('Country').sort_values('Positive Percentage', ascending=False).head(30)
pos_rate

,New Case,New Test,Positive Percentage
Country,,,
Mexico,835,2135,39.11
Afghanistan,172,604,28.48
Qatar,957,3420,27.98
Panama,241,1242,19.40
Honduras,34,186,18.28
USA,756,5557,13.60
Belarus,826,6320,13.07
Bangladesh,497,3812,13.04
Jamaica,45,394,11.42


# NYT US data

## County level

In [452]:
df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')

df.to_csv('data/Country detailed/Detailed Data - US - NYT.csv')
df.iloc[-1]['date']

'2020-04-26'

## State level (cumulative for Tableau)

In [422]:
df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv')
df = df.pivot(index='date',columns='state',values='cases').iloc[32:]
df = df[df.sum().sort_values(ascending=False).index]
# for i in list(df):
#     df[i] = df[i] - df[i].shift(1)
df.to_csv('data/Country detailed/US by state.csv')
df.index[-1]

# Generate daily increase of yesterday for tableau
for i in list(df):
    df[i] = df[i] - df[i].shift(1)
df.iloc[-1:,:].reset_index(drop=True).to_csv('data/Country detailed/US_yesterday_by_state.csv')

## State level (daily new for dynamics)

In [423]:
df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv')
df = df.pivot(index='date',columns='state',values='cases').iloc[32:]
df = df[df.iloc[-1].sort_values(ascending=False).index]
for i in list(df):
    df[i] = df[i] - df[i].shift(1)
df.index.rename('US', inplace=True)
df.to_clipboard()